In [54]:
import requests
import json
import pandas as pd
import os

In [56]:
os.chdir('C:\\Users\\Yingting-PC\\Yingting Python Notebooks\\Yelp api')

In [60]:
neig_name=pd.read_csv('Neighborhoods.csv')
neig_name.shape

(98, 1)

In [108]:
with open('api key.txt') as f:
    YELP_API_KEY=f.readline()

In [98]:
# https://www.yelp.com/developers/documentation/v3/business_search
YELP_SEARCH='https://api.yelp.com/v3/businesses/search'
headers={'Authorization': 'Bearer %s' % YELP_API_KEY}

In [99]:
columns=['neighborhood','name','distance','categories','address','display_phone','price','rating',
        'review_count','url','latitude','longitude']
final = pd.DataFrame(columns=columns)

for neighborhood in neig_name.Name:
    params = {'location': neighborhood + ', Chicago, IL', 'term': 'Restaurant','sort_by': 'review_count','limit': 50}
    query=requests.get(YELP_SEARCH,headers=headers,params=params)
    df=pd.DataFrame.from_dict(query.json()['businesses'])
    
    if 'id' in df:
        df['categories']=df.categories.apply(lambda x: x[0]['title'])
        df['latitude']=df.coordinates.apply(lambda x: x['latitude'])
        df['longitude']=df.coordinates.apply(lambda x: x['longitude'])
        df['address']=df.location.apply(lambda x: x['display_address'][0])
        df['neighborhood']=neighborhood
    
    final = pd.concat([final,df[columns]],ignore_index=True)

In [107]:
final.to_csv('Chicago Restaurants.csv',index=False)